## Forward Modeling

The goal of this notebook is to "illustrate how users might do forward-model simulations of a dispersed 2-D grism spectrum from a one-D model input spectrum." This is currently possible at least with the Grizli package and SBE, and presumably aXeSIM, although word on the street is that there is currently only one individual who knows how to/has gotten aXeSIM working.

#### AstroGrism Potential Improvements

So far, I'm under the impression that the AstroGrism project can potentially improve the state of grism modeling in a few areas:

    - Ease of installation: the environment for the aXe cookbook, for example, has many package dependencies and is frustrating to get set up (the instructions out of the box don't work). Grizli is in a better state but still has dependencies which may cause installation issues (GEOS+shapely for example) but aren't needed for this particular use.
    - Trimming complexity/dependencies: following from the first point, existing packages seem to take an "everything and the kitchen sink" approach. We could trim to a lighter, more focused set of capabilities and ditch many large underlying code bases that are only used in one or two places.
    - Ease of use: a more focused set of tools with better modularity might be attractive for users.
    - Ease of maintenance: trimming dependencies and refactoring would likely make the code easier to maintain than existing packages. Grizli for example has underlying Cython code that may be able to be ditched without severe performance hits.
    - Performance (maybe!): looking at some grizli source code (e.g. disperse.pyx) I see a lot of loops - there might be things that would be faster if vectorized.

#### Pseudo code

An easy 2D spectrum modeling process for the user might look something like this:
```
  import astrogrism as ag
  from specutils import Spectrum1D
  
  target = [direct image of object]
  # Load a configuration file containing information about the grism
  grism_config = ag.load_config(filename/instrument)
  # Load a 1D spectrum for the object
  spec1d = Spectrum1D.read(spectrum file)
  # Model the dispersed 2D spectrum of the object
  spec2d = ag.disperse_spectrum(target, grism_config, spectrum_1d=spec1d)
```

### Grizli

Grizli contains a `GrismDisperser` class in its `model` module that handles doing forward transformations. This object takes a grismconf files and stores as attributes all of the information needed to calculate the dispersion. It also can take as input a direct image and segmentation map (as well as an ID corresponding to the object of interest in the segmentation map). The user can then use `GrismDisperser.compute_model()` to generate a model 2D spectrum given (all optional) a segment ID, thumbnail direct image, and 1D spectrum. If no 1D spectrum is given a flat spectrum is assumed. 

The actual dispersion calculation, once `compute_model` does all the prep, is done by `grizli.utils_c.disperse.disperse_grism_object`, which appears to be Cython code (I'm unfamiliar with Cython).